## Pyvis, Gephi

In [3]:
import sys
sys.path.append("..")
from rcn_py import data_storage
from rcn_py import visual
import networkx as nx
from pyvis.network import Network


In [7]:
# pyvis
def pyvis_network_by_datafile(fullname, folderpath, outputpath):
    node_data = data_storage.assign_group_node(fullname, folderpath)
    edge_data, link = data_storage.get_links_from_csv(fullname, folderpath)
    all_node_in_link = edge_data['source'].values.tolist()+ edge_data['target'].values.tolist()
    
    # getting weights of links
    weights = []
    for i in link:
        weights.append(link.count(i))
    edge_data['weight'] = weights
    
    # getting a group id
    groups = node_data.groupby("group")["orcid"].apply(list).reset_index()
    groups["group"] = groups.index
    
    # finding group id for each node from groups dataframe
    nodes = node_data.merge(groups, how="left", on=["group"])
    nodes["title"] = node_data[["name"]]
    nodes["label"] = node_data[["name"]]
    nodes = nodes.drop("orcid_y", axis=1).set_index("orcid_x")

    colors = ['#1f78b4', '#e08a34', '#643b8a', '#3d7a36', '#a34b43', '#47a68b', 'blue', 'red', 'green', 'black']
    node_color = [colors[n] for n in nodes['group']]
    nodes['color'] = node_color
    # the size of the node could depend on the value, just a test
    nodes['value'] = [all_node_in_link.count(n) for n in node_data['orcid']]

    # Pyvis network
    N = Network(height=800, width="100%", bgcolor="#222222", font_color="white")
    N.toggle_hide_edges_on_drag(False)
    N.barnes_hut()

    # collecting node attributes for network x
    #node_attrs = nodes.to_dict("index")

    # creating a network x graph from dataframes
    graph = nx.from_pandas_edgelist(edge_data, edge_attr=True)

    d = dict(graph.degree)
    d.update((x, 100*y) for x, y in d.items())
    nodes['value'] = d.values()

    # collecting node attributes for network x
    node_attrs = nodes.to_dict("index")
    nx.set_node_attributes(graph, node_attrs)

    d = dict(graph.degree)
    d.update((x, 100*y) for x, y in d.items())
    nx.set_node_attributes(graph, d, 'size')

    lastname = fullname.split(' ')[-1]
    N.from_nx(graph)
    N.show(lastname+'_datafile.html')

In [116]:
#gephi
def gephi_network_by_datafile(fullname, folderpath, outputpath):
    node_data = data_storage.assign_group_node(fullname, folderpath)
    edge_data, link = data_storage.get_links_from_csv(fullname, folderpath)
    weights = []
    
    all_node_in_link = edge_data['source'].values.tolist()+ edge_data['target'].values.tolist()
    for i in link:
        weights.append(link.count(i))
    edge_data['weight'] = weights

    edges = zip(edge_data['source'], edge_data['target'], weights)  

    # getting a group id
    groups = node_data.groupby("group")["orcid"].apply(list).reset_index()
    groups["group"] = groups.index
    
    # finding group id for each node from groups dataframe
    nodes = node_data.merge(groups, how="left", on=["group"])
    nodes["title"] = node_data[["name"]]
    nodes["label"] = node_data[["name"]]
    nodes = nodes.drop("orcid_y", axis=1).set_index("orcid_x")

    # setting color for each group 
    colors = ['#1f78b4', '#e08a34', '#643b8a', '#3d7a36', '#a34b43', '#47a68b', 'blue', 'red', 'green', 'black']
    node_color = [colors[n] for n in nodes['group']]
    nodes['color'] = node_color

    # the size of the node could depend on the value, just a test
    nodes['value'] = [all_node_in_link.count(n) for n in node_data['orcid']]

    # collecting node attributes for network x
    node_attrs = nodes.to_dict("index")
    
    # creating a network x graph from dataframes
    G = nx.from_pandas_edgelist(edge_data, edge_attr='weight')
    nx.set_node_attributes(G, node_attrs)

    # ...
    # graph = nx.from_pandas_edgelist(edge_data, edge_attr=True)
    # nx.set_node_attributes(graph, node_attrs)

    lastname = fullname.split(' ')[-1]
    out_path = outputpath + '/' + lastname + '.gexf'
   
    nx.write_gexf(G,out_path)


In [8]:
pyvis_network_by_datafile('Stefan Verhoeven', "/Users/jennifer/RCSNA/data", "/Users/jennifer/RCSNA/visualization")

/Users/jennifer/RCSNA/rcn-py/notebook/../rcn_py/data_storage.py:206: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  authors_orcid[i] = authors_orcid[i].strip('[')
/Users/jennifer/RCSNA/rcn-py/notebook/../rcn_py/data_storage.py:207: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  authors_orcid[i] = authors_orcid[i].strip(']')
/Users/jennifer/RCSNA/rcn-py/notebook/../rcn_py/data_storage.py:208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  

In [4]:
visual.gephi_network_by_datafile('Stefan Verhoeven', "/Users/jennifer/RCSNA/data", "/Users/jennifer/RCSNA/visualization")

/Users/jennifer/RCSNA/rcn-py/notebook/../rcn_py/data_storage.py:206: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  authors_orcid[i] = authors_orcid[i].strip('[')
/Users/jennifer/RCSNA/rcn-py/notebook/../rcn_py/data_storage.py:207: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  authors_orcid[i] = authors_orcid[i].strip(']')
/Users/jennifer/RCSNA/rcn-py/notebook/../rcn_py/data_storage.py:208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  

In [22]:
node_data = data_storage.assign_group_node('Stefan Verhoeven', "/Users/jennifer/RCSNA/data")
edge_data, link = data_storage.get_links_from_csv('Stefan Verhoeven', "/Users/jennifer/RCSNA/data")
weights = []
    
all_node_in_link = edge_data['source'].values.tolist()+ edge_data['target'].values.tolist()
for i in link:
        weights.append(link.count(i))
edge_data['weight'] = weights

edges = zip(edge_data['source'], edge_data['target'], weights)  

# getting a group id
groups = node_data.groupby("group")["orcid"].apply(list).reset_index()
groups["group"] = groups.index
    
# finding group id for each node from groups dataframe
nodes = node_data.merge(groups, how="left", on=["group"])
nodes["title"] = node_data[["name"]]
nodes["label"] = node_data[["name"]]
nodes = nodes.drop("orcid_y", axis=1).set_index("orcid_x")




/Users/jennifer/RCSNA/rcn-py/notebook/../rcn_py/data_storage.py:206: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  authors_orcid[i] = authors_orcid[i].strip('[')
/Users/jennifer/RCSNA/rcn-py/notebook/../rcn_py/data_storage.py:207: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  authors_orcid[i] = authors_orcid[i].strip(']')
/Users/jennifer/RCSNA/rcn-py/notebook/../rcn_py/data_storage.py:208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  

In [21]:
len(node_data)

89

In [23]:

# creating a network x graph from dataframes
graph = nx.from_pandas_edgelist(edge_data, edge_attr=True)
#nx.set_node_attributes(graph, node_attrs)

d = dict(graph.degree)
d.update((x, 100*y) for x, y in d.items())

#nx.set_node_attributes(graph, d, 'size')

nodes['value'] = d.values()

node_attrs = nodes.to_dict("index")

In [24]:
node_attrs

{'0000-0001-9795-7981': {'name': 'Niels Drost',
  'group': 0,
  'title': 'Niels Drost',
  'label': 'Niels Drost',
  'value': 2300},
 '0000-0002-5697-5697': {'name': 'Rolf Hut',
  'group': 0,
  'title': 'Rolf Hut',
  'label': 'Rolf Hut',
  'value': 1600},
 '0000-0002-7200-3353': {'name': 'Nick Van De Giesen',
  'group': 0,
  'title': 'Nick Van De Giesen',
  'label': 'Nick Van De Giesen',
  'value': 1600},
 '0000-0002-7508-3272': {'name': 'Ben van Werkhoven',
  'group': 0,
  'title': 'Ben van Werkhoven',
  'label': 'Ben van Werkhoven',
  'value': 1600},
 '0000-0003-0157-4818': {'name': 'Jerom P.M. Aerts',
  'group': 0,
  'title': 'Jerom P.M. Aerts',
  'label': 'Jerom P.M. Aerts',
  'value': 1600},
 '0000-0002-8928-7831': {'name': 'Jaro Camphuijsen',
  'group': 0,
  'title': 'Jaro Camphuijsen',
  'label': 'Jaro Camphuijsen',
  'value': 1600},
 '0000-0002-8024-0412': {'name': 'Inti Pelupessy',
  'group': 0,
  'title': 'Inti Pelupessy',
  'label': 'Inti Pelupessy',
  'value': 1600},
 '0000-

In [ ]:

d = dict(graph.degree)
d.update((x, 100*y) for x, y in d.items())